# Mexico Drug Traffic Activities 

## Librerie Usate

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import itertools
from pyvis.network import Network
import os
import shutil
import collections
import random
import numpy as np

Inserire percorso della propria cartella

In [ ]:
percorso_cartella = "/Users/emanuele/Desktop/SNA/PROGETTO"

## Dataset

In [ ]:
dataset = pd.read_csv(percorso_cartella+'/DATASETS/CosciaRios2012_DataBase.csv') #lettura del dataset
dataset.drop('Code', axis=1, inplace=True) #rimuoviamo la collona dove sono presenti i codici "provinciali"
dataset = dataset.groupby(['State','Year']).sum() #sommiamo le nostre istanze in base a Stato e Anno
dataset.to_csv('DATASETS/Mexican_Drug_Cartels_dta.csv') #salviamo il dataset

In [ ]:
dataset = pd.read_csv(percorso_cartella+'/DATASETS/Mexican_Drug_Cartels_dta.csv')
#dividiamo il nostro datastet in due decadi 
dta_9000 = dataset[dataset.Year < 2000] #1990-1999
dta_0010 = dataset[dataset.Year >= 2000 ] #2000-2010

## Funzioni

In [ ]:
def Edges_Generator(dataset, name_txt, Qcartelli, stateL=False, yearL=False, yearXstate=False, cartelL=False,
                    link_state_year=False):
    #permette la creazione di edges partendo dal dataset di partenza

    #esegue un check per vedere crearli su tutte le famiglie o su una in particolare
    if Qcartelli == 'ALL':
        cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
                    'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']
    else:
        cartelli = []
        cartelli.append(Qcartelli)

    res_list = []
    
    for state in dataset.State.unique(): #creiamo un subset per ogni stato in maniera ricorsiva
        sub1 = dataset[dataset.State == state]

        for year in sub1.Year: #creiamo un subset per ogni anno in maniera ricorsiva
            sub2 = sub1[sub1.Year == year]
            
            for single in cartelli:
                value = sub2[single].iloc[0] #troviamo il numero di volte che il cartello è apparso
                list = []

                if value > 0 : #eseguiamo un controllo per sapere se creare o no il collegamento
                    #in base alle impostazioni decise prendiamo PARTENZA(A), ARRIVO(B) e WEIGHT(W)
                    if yearXstate == True:
                        year_ = str(year)
                        _state = str(state)
                        year_state = str(year_ + '_' + _state)

                    if stateL == True:
                        list.append(state)
                    
                    if yearL == True:
                        list.append(year)
                    elif yearXstate == True:
                        list.append(year_state)
                    
                    if Qcartelli == 'ALL':
                        if cartelL == True:
                            list.append(single)
                    else:
                        if cartelL == True:
                            list.append(Qcartelli)
                    
                    if link_state_year == True:
                        list.append(1)
                    else:
                        list.append(value)
                    res_list.append(list)

    #creiamo il file.txt con il nome assegnato                   
    name_file = str(name_txt + ".txt")
    file = open(name_file, "w+")

    file.write('A'+',')
    file.write('B'+',')
    file.write('Weight'+'\n')

    for list in res_list:
        time = 0
        for element in list:
            time = time + 1
            e = str(element)
            file.write(e)
            if time < 3:
                file.write(',')
            else:
                file.write('\n')

    file.close()

    return(file)

#----------------------------------------------------------------------------------------#

def Edges_Generator_for_Cartel(name_cartel, cartel_sigla, years_9000=False, years_0010=False,street=percorso_cartella):
    #permette la creazione di edges per i singoli cartelli

    if years_9000 == True: #svolgo operazioni per gli anni 1990 - 1999
        #assegno i nomi ai file tramite una concatenazione di stringhe
        name_txt_9000_yXc = 'year_'+str(name_cartel)+'_'+str(cartel_sigla)+'9000'
        name_txt_9000_sXy = 'state_year_'+str(name_cartel)+'_'+str(cartel_sigla)+'9000'
        name_csv_9000 = str(name_cartel)+'_'+'9000'+'.csv'

        #utilizzo la funzione creata in precesenza per generare gli edges
        Edges_Generator(dta_9000,name_txt_9000_yXc, name_cartel, yearL=True,cartelL=True)
        Edges_Generator(dta_9000,name_txt_9000_sXy, name_cartel, stateL=True, yearL=True,link_state_year=True)

        name_txt_9000_yXc = name_txt_9000_yXc + '.txt'
        name_txt_9000_sXy = name_txt_9000_sXy + '.txt'

        #creo i percorsi delle per leggeere i file.txt sopra creati
        percorso_cartella = street + '/'
        percorso_1 = str(percorso_cartella) + str(name_txt_9000_sXy)
        percorso_2 = str(percorso_cartella) + str(name_txt_9000_yXc)
        percorso_3 = str(percorso_cartella) + str(name_csv_9000)

        df_1 = pd.read_csv(str(percorso_1))
        df_2 = pd.read_csv(str(percorso_2))
        df_2 = df_2.groupby(['A','B']).sum() #effettuo una somma in base ad anno e cartello
        df_2.to_csv(name_csv_9000) 
        df_2 = pd.read_csv(percorso_3)
        df = pd.concat([df_1,df_2],ignore_index=True, sort=False) #unisco le due liste di edges
    
    elif years_0010 == True: #svolgo le stesse operazioni per gli anni fra 2000 e 2010
        name_txt_0010_yXc = 'year_'+str(name_cartel)+'_'+str(cartel_sigla)+'0010'
        name_txt_0010_sXy = 'state_year_'+str(name_cartel)+'_'+str(cartel_sigla)+'0010'
        name_csv_0010 = str(name_cartel)+'_'+'0010'+'.csv'

        Edges_Generator(dta_0010,name_txt_0010_yXc, name_cartel, yearL=True,cartelL=True)
        Edges_Generator(dta_0010,name_txt_0010_sXy, name_cartel, stateL=True, yearL=True,link_state_year=True)

        name_txt_0010_yXc = name_txt_0010_yXc + '.txt'
        name_txt_0010_sXy = name_txt_0010_sXy + '.txt'

        percorso_cartella = street + '/'
        percorso_1 = str(percorso_cartella) + str(name_txt_0010_sXy)
        percorso_2 = str(percorso_cartella) + str(name_txt_0010_yXc)
        percorso_3 = str(percorso_cartella) + str(name_csv_0010)

        df_1 = pd.read_csv(str(percorso_1))
        df_2 = pd.read_csv(str(percorso_2))
        df_2 = df_2.groupby(['A','B']).sum()
        df_2.to_csv(name_csv_0010)
        df_2 = pd.read_csv(percorso_3)
        df = pd.concat([df_1,df_2],ignore_index=True, sort=False) 

    return(df)

#----------------------------------------------------------------------------------------#

def Edges_Only_Cartel_Network_Start(dataset,name_txt):
    
    cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
                'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']

    final_list = []
    for state in dataset.State.unique():
        sub1 = dataset[dataset.State == state]

        for year in sub1.Year:
            sub2 = sub1[sub1.Year == year]
            
            for single in cartelli:
                value = sub2[single].iloc[0]
                list = []
                if value > 0 :
                    list.append(state)
                    list.append(year)
                    list.append(single)
                    list.append(value)
                    final_list.append(list)
                    
    name_file = str(name_txt + ".txt")
    file = open(name_file, "w+")

    file.write('A'+',')
    file.write('Bridge'+',')
    file.write('B'+',')
    file.write('Weight'+'\n')

    for list in final_list:
        time = 0
        for element in list:
            time = time + 1
            e = str(element)
            file.write(e)
            if time < 4:
                file.write(',')
            else:
                file.write('\n')

    file.close() 

    return(file)

#----------------------------------------------------------------------------------------#

def Edges_Only_Cartel_Network_Final(dataset, name_txt, name_csv, year_selected, street=percorso_cartella):

    Edges_Only_Cartel_Network_Start(dataset,name_txt)

    name_file_txt = name_txt+'.txt'
    name_file_csv = name_csv+'.csv'
    percorso_cartella = street
    percorso1 = percorso_cartella + name_file_txt
    percorso2 = percorso_cartella + name_file_csv

    data =  pd.read_csv(percorso1)

    data = data[data.Bridge == year_selected]

    data_AB = data[['A','B','Weight']]
    data_AB = data_AB.groupby(['A','B']).sum()

    data_AB.to_csv(name_file_csv)

    df = pd.read_csv(percorso2)

    edges_list = []

    for state in df.A.unique():
        sub = df[df.A == state]
        
        if len(sub) > 1:
            list_cartels = []
            for cartel in sub.B:
                list_cartels.append(cartel)

            for v in itertools.combinations(list_cartels, 2):
                edges_list.append(v)
                
                
    new_df = pd.DataFrame(columns=['A','B'])
    new_df[['A','B']]=edges_list

    new_cols = ['A','B']
    new_df = new_df.reindex(new_df.columns.union(new_cols), axis=1) 

    return(new_df)

#----------------------------------------------------------------------------------------#

def Network_Cartels_Meeting_Generator(dataset,Graph,colore):
    
    cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
            'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']
    
    edges_cartels = dataset.values.tolist()

    for cartello in cartelli:
        name = str(cartello)
        Graph.add_node(name, color = colore)

    for i in range(len(edges_cartels)):
        edge = edges_cartels[i]
        add = -1

        for o in range(len(edge)):
            add = add + 1
            
            if add == 0:
                A = edge[add]
                
            elif add == 1:
                B = edge[add]
            elif add == 2:
                W = edge[add]

        Graph.add_edge(str(A), str(B) ,weight=int(W))
          
    return(Graph)   

#----------------------------------------------------------------------------------------#

def metrics(network, link_select=False, nodi_select=False, degree_select=False, density_select=False, matrix_select=False,
            betweenness_select=False, closeness_select=False, eigen_select=False, all_metrics=False):

    if all_metrics == True:
        link_select = True
        nodi_select = True
        degree_select = True
        density_select = True
        matrix_select = True
        betweenness_select = True 
        closeness_select = True
        eigen_select = True

    #CONTO DEGLI EDGE/LINK DELLA RETE 
    if link_select == True:
        l = network.number_of_edges()
        print('Gli edge della rete sono:', l)
    
    #CONTO I NODI DELLA RETE
    if nodi_select == True:
        nodi = network.number_of_nodes()
        print('#','-'*60,'#')
        print('I nodi della rete sono:',nodi)

    #DEGREE DEI NODI 
    if degree_select == True:
        print('#','-'*60,'#')
        list = []
        for node in network.degree():
            list.append(node)
        nodi_degree = pd.DataFrame(list, columns = ['Node', 'Degree'])
        nodi_degree.to_csv('nodi_degree.csv')
        

        somma_degree = sum(dict(network.degree()).values())
        print('La somma dei node degree è:',somma_degree)
        print('\nL\'average degree di questa network è:',somma_degree/(len(network)))
        print('\nI valori degree maggiori sono:', '\n', nodi_degree.nlargest(5, 'Degree'))
        print('\nI restanti valori sono stati salvati nel file \'nodi_degree.csv\'')
    
    #DENSITA'
    if density_select == True:
        print('#','-'*60,'#')
        print('Densità della rete equivale a:',nx.density(network))

    #MATRICE DI ADIACENZA PESATA
    if matrix_select == True:
        print('#','-'*60,'#')
        print('La Matrice di adiacenza pesata è stata salvata correttamente.') 
        m = nx.to_pandas_adjacency(network, nodelist=sorted(network.nodes), dtype=int)
        m.to_excel('adjacency_matrix.xlsx')

    #BETWEENNESS CENTRALITY
    if betweenness_select == True:
        B = nx.betweenness_centrality(network,normalized=False)
        print('#','-'*60,'#')
        list_bet = []
        for value in B.items():
            list_bet.append(value)
        bet_value = pd.DataFrame(list_bet, columns = ['Node', 'Values'])
        bet_value.to_csv('Betweenness_value.csv')
        print('I valori della Betweenness Centrality maggiori sono:', '\n', bet_value.nlargest(5, 'Values'))
        print('\nI restanti valori sono stati salvati nel file \'Betweenness_value.csv\'')
    
    #CLOSENESS CENTRALITY
    if closeness_select == True:
        C = nx.closeness_centrality(network)
        print('#','-'*60,'#')
        list_clo = []
        for value in C.items():
            list_clo.append(value)
        clo_value = pd.DataFrame(list_clo, columns = ['Node', 'Values'])
        clo_value.to_csv('Closeness_value.csv')
        print('I valori della Closeness Centrality maggiori sono:', '\n', clo_value.nlargest(5, 'Values'))
        print('\nI restanti valori sono stati salvati nel file \'Closeness_value.csv\'')

    
    #EIGENVECTOR CENTRALITY
    if eigen_select == True:
        eigen = nx.eigenvector_centrality(network)
        print('#','-'*60,'#')
        list_eigen = []
        for value in eigen.items():
            list_eigen.append(value)
        eig_value = pd.DataFrame(list_eigen, columns = ['Node', 'Values'])
        eig_value.to_csv('Eigenvector_value.csv')
        print('I valori dell\'Eigenvector Centrality maggiori sono:', '\n', eig_value.nlargest(5, 'Values'))
        print('\nI restanti valori sono stati salvati nel file \'Eigenvector_value.csv\'')

#----------------------------------------------------------------------------------------#

def BiGram_Generator_Counter(dataset):

    dataset['to'] = '-'
    add = dataset.A +dataset.to+ dataset.B
    dataset['bi-grams'] = add

    list_ = add.values.tolist()

    frequency = collections.Counter(list_)

    list_f = list(frequency.items())

    cartels_connections = []
    W = []
    A = []
    B = []
    for i in range(len(list_f)):
        cartels_connections.append(list_f[i][0])
        W.append(list_f[i][1])
        A.append(cartels_connections[i].split('-')[0])
        B.append(cartels_connections[i].split('-')[1])
    
    final = pd.DataFrame()
    final['A'] = A
    final['B'] = B
    final['W'] = W
    
    return(final)

#----------------------------------------------------------------------------------------#

def Network_Generator_For_Cartels(Graph,state_year_csv,df_cartel,name_cartel):

    state_year = state_year_csv
    for a in state_year.A.unique():
        name = str(a)
        Graph.add_node(name, color = 'green')

    for b in state_year.B.unique():
        name = str(b)
        Graph.add_node(name, color = 'blue')

    Graph.add_node(name_cartel, color = 'red')

    list_df = df_cartel.values.tolist()
    f = len(list_df)

    for s in range(f):
        edge = list_df[s]
        add = -1

        for x in range(len(edge)):
            add = add + 1
            
            if add == 0:
                A = edge[add]
                
            elif add == 1:
                B = edge[add]
                
            elif add == 2 :
                W = edge[add]

        Graph.add_edge(str(A), str(B) ,weight=int(W))

    return(Graph)

#----------------------------------------------------------------------------------------#

def html_network(net,data,yearXstate_txt): 
    cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
                'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']

    yearXstate = yearXstate_txt
    stati = yearXstate.A.unique().astype(str)
    yearXstate = yearXstate.B.unique()

    net.barnes_hut()
    data_ = data

    sources = data_['B']
    targets = data_['A']
    weights = data_['Weight']

    edge_data = zip(sources, targets, weights)

    for cartel in cartelli:
        net.add_node(cartel,color='red',color_border='black')

    for stato in stati:
        net.add_node(stato,color='green')

    for y_s in yearXstate:
        net.add_node(y_s,color='blue')

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]
        
        net.add_node(dst, dst, title=dst)
        net.add_edge(src, dst, value=w,opacity=0.1)

    neighbor_map = net.get_adj_list()

    net.repulsion(node_distance=200, spring_length=80)

    for node in net.nodes:
        node['value'] = len(neighbor_map[node['id']])

    net.show_buttons(filter_=True)
    return(net)

## ANALISI DELLE NETWORK

### Analisi sulla Network Generale

Andiamo a creare la lista degli edges per ambe le decadi per formare le reti per l'analisi. 

In partcolare creeremo: 
* (anno diversificato per stato, cartello)
* (stato, anno divesificato per stato)

In [ ]:
Edges_Generator(dta_9000,'G_yearXstate_cartel_9000','ALL',yearXstate=True,cartelL=True) 
Edges_Generator(dta_9000,'G_state_yearXstate_9000','ALL',stateL=True, yearXstate=True)
df_1 = pd.read_csv(percorso_cartella+'/G_state_yearXstate_9000.txt') #leggiamo i file di testo come dataset 
df_2 = pd.read_csv(percorso_cartella+'/G_yearXstate_cartel_9000.txt') #leggiamo i file di testo come dataset 
df_9000 = pd.concat([df_1,df_2],ignore_index=True, sort=False) #concateniamo gli oggetti lungo l'asse verticale  

Edges_Generator(dta_0010,'G_yearXstate_cartel_0010','ALL',yearXstate=True,cartelL=True) 
Edges_Generator(dta_0010,'G_state_yearXstate_0010','ALL',stateL=True, yearXstate=True) 
df_1 = pd.read_csv(percorso_cartella+'/G_state_yearXstate_0010.txt') #leggiamo i file di testo come dataset  
df_2 = pd.read_csv(percorso_cartella+'/G_yearXstate_cartel_0010.txt') #leggiamo i file di testo come dataset 
df_0010 = pd.concat([df_1,df_2],ignore_index=True, sort=False) #concateniamo gli oggetti lungo l'asse verticale  

In [ ]:
#trasformiamo la colonna A in stringa per non avere problemi con le operazioni successive
df_9000['A'] = df_9000.A.astype(str) 
df_0010['A'] = df_0010.A.astype(str)

Procediamo con la creazione delle Network per poterle visualizzare. 
Per la creazione e la visualizazione useremo: 
* NetworkX
* pivis

In [ ]:
net_9000 = Network(height='750px', width='100%', bgcolor='white', font_color='black') #creiamo la network con pyvis
yearXstate_9000= pd.read_csv(percorso_cartella+'/G_state_yearXstate_9000.txt')
html_network(net_9000,df_9000,yearXstate_9000)
net_9000.show('networkDTO_9000.html') #crea il file html da visualizzare in un browser

In [ ]:
net_0010 = Network(height='750px', width='100%', bgcolor='white', font_color='black') #creiamo la network con pyvis
yearXstate_0010 = pd.read_csv(percorso_cartella+'/G_state_yearXstate_0010.txt')
html_network(net_0010,df_0010,yearXstate_0010)
net_0010.show('networkDTO_0010.html') #crea il file html da visualizzare in un browser 

In [ ]:
#Creazione della Network con NetworkX per poter calcolare le metriche
netX_9000 = nx.Graph()

netX_data_9000 = df_9000

cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
            'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']

stati = yearXstate_9000.A.unique()
yearXstate_9000 = yearXstate_9000.B.unique()

for a in stati:
    name = str(a)
    netX_9000.add_node(name, color = 'red')

for b in yearXstate_9000:
    name = str(b)
    netX_9000.add_node(name, color = 'blue')

for cartel in cartelli:
    netX_9000.add_node(cartel, color = 'green')

list_df = netX_data_9000.values.tolist()
f = len(list_df)

for s in range(f):
    edge = list_df[s]
    add = -1

    for x in range(len(edge)):
        add = add + 1
        
        if add == 0:
            A = edge[add]
            
        elif add == 1:
            B = edge[add]
            
        elif add == 2 :
            W = edge[add]

    netX_9000.add_edge(str(A), str(B) ,weight=int(W))

In [ ]:
#Creazione della Network con NetworkX per poter calcolare le metriche
netX_0010 = nx.Graph()

netX_data_0010 = df_0010

cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
            'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']

stati = yearXstate_0010.A.unique()
yearXstate_0010= yearXstate_0010.B.unique()

for a in stati:
    name = str(a)
    netX_0010.add_node(name, color = 'red')

for b in yearXstate_0010:
    name = str(b)
    netX_0010.add_node(name, color = 'blue')

for cartel in cartelli:
    netX_0010.add_node(cartel, color = 'green')

list_df = netX_data_0010.values.tolist()
f = len(list_df)

for s in range(f):
    edge = list_df[s]
    add = -1

    for x in range(len(edge)):
        add = add + 1
        
        if add == 0:
            A = edge[add]
            
        elif add == 1:
            B = edge[add]
            
        elif add == 2 :
            W = edge[add]

    netX_0010.add_edge(str(A), str(B) ,weight=int(W))

In [ ]:
#calcoliamo le metriche per la rete generale della prima decade (1990-1999)
metrics(netX_9000,link_select=True, nodi_select=True, degree_select=True, density_select=True, matrix_select=True,
betweenness_select=True) 

In [ ]:
#calcoliamo le metriche per la rete generale della prima decade (2000-2010)
metrics(netX_0010,link_select=True, nodi_select=True, degree_select=True, density_select=True, matrix_select=True,
betweenness_select=True)

## PER FAMIGLIA

In [ ]:
#1990-1999: in alcuni non sono presenti dati quindi decidiamo di non eseguire la creazione 
#df_BL9000 = Edges_Generator_for_Cartel('Beltran_Leyva','BL',years_9000=True)
#df_BLF9000 = Edges_Generator_for_Cartel('Beltran_Leyva_Family','BLF',years_9000=True)
#df_F9000 = Edges_Generator_for_Cartel('Familia','F',years_9000=True)
df_G9000 = Edges_Generator_for_Cartel('Golfo','G',years_9000=True)
df_J9000 = Edges_Generator_for_Cartel('Juarez','J',years_9000=True)
df_S9000 = Edges_Generator_for_Cartel('Sinaloa','S',years_9000=True)
#df_SF9000 = Edges_Generator_for_Cartel('Sinaloa_Family','SF',years_9000=True)
df_T9000 = Edges_Generator_for_Cartel('Tijuana','T',years_9000=True)
df_Z9000 = Edges_Generator_for_Cartel('Zetas','Z',years_9000=True)
#df_O9000 = Edges_Generator_for_Cartel('Otros','O',years_9000=True)

In [ ]:
#2000-2010
df_BL0010 = Edges_Generator_for_Cartel('Beltran_Leyva','BL',years_0010=True)
df_BLF0010 = Edges_Generator_for_Cartel('Beltran_Leyva_Family','BLF',years_0010=True)
df_F0010 = Edges_Generator_for_Cartel('Familia','F',years_0010=True)
df_G0010 = Edges_Generator_for_Cartel('Golfo','G',years_0010=True)
df_J0010 = Edges_Generator_for_Cartel('Juarez','J',years_0010=True)
df_S0010 = Edges_Generator_for_Cartel('Sinaloa','S',years_0010=True)
df_SF0010 = Edges_Generator_for_Cartel('Sinaloa_Family','SF',years_0010=True)
df_T0010 = Edges_Generator_for_Cartel('Tijuana','T',years_0010=True)
df_Z0010 = Edges_Generator_for_Cartel('Zetas','Z',years_0010=True)
df_O0010 = Edges_Generator_for_Cartel('Otros','O',years_0010=True)

### Golfo

In [ ]:
net_golfo_9000 = nx.Graph()
sXy_golfo_9000 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Golfo_G9000.txt')
Network_Generator_For_Cartels(net_golfo_9000,sXy_golfo_9000,df_G9000,'Golfo')

In [ ]:
nt = Network('1000px', '1000px')
nt.from_nx(net_golfo_9000, default_edge_weight=3,show_edge_weights=True)
nt.show('golfo_9000.html')

In [ ]:
all_weights=[]
for (node1,node2,data) in net_golfo_9000.edges(data=True):
        all_weights.append(data['weight']) 

nodes_color=[]
for (node1,data) in net_golfo_9000.nodes(data=True):
        nodes_color.append(data['color']) 


unique_weights = list(set(all_weights))

pos = nx.spring_layout(net_golfo_9000,seed=1234)

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in net_golfo_9000.edges(data=True) if edge_attr['weight']==weight]
        width = weight
        nx.draw_networkx_edges(net_golfo_9000,pos,edgelist=weighted_edges,width=width)

nx.draw_networkx_nodes(net_golfo_9000,pos,node_color=nodes_color, node_size=4000)
nx.draw_networkx_labels(net_golfo_9000,pos,font_size=15, font_family="sans-serif")
plt.rcParams["figure.figsize"] = (25,25)

pass

In [ ]:
metrics(net_golfo_9000,all_metrics=True)

In [ ]:
net_golfo_0010 = nx.Graph()
sXy_golfo_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Golfo_G0010.txt')
Network_Generator_For_Cartels(net_golfo_0010,sXy_golfo_0010,df_G0010,'Golfo')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_golfo_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('golfo_0010.html')

In [ ]:
metrics(net_golfo_0010,all_metrics=True)

### Beltran Leyva 

In [ ]:
net_bl_0010 = nx.Graph()
sXy_bl_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Beltran_Leyva_BL0010.txt')
Network_Generator_For_Cartels(net_bl_0010,sXy_bl_0010,df_BL0010,'Beltran_Leyva')

In [ ]:
metrics(net_bl_0010,all_metrics=True)

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_bl_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('bl_0010.html')

### Beltran Leyva Family

In [ ]:
net_blf_0010 = nx.Graph()
sXy_blf_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Beltran_Leyva_Family_BLF0010.txt')
Network_Generator_For_Cartels(net_blf_0010,sXy_blf_0010,df_BLF0010,'Beltran_Leyva_Family')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_blf_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('blf_0010.html')

### Familia

In [ ]:
net_familia_0010 = nx.Graph()
sXy_familia_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Familia_F0010.txt')
Network_Generator_For_Cartels(net_familia_0010,sXy_familia_0010,df_F0010,'Familia')

In [ ]:
metrics(net_familia_0010,all_metrics=True)

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_familia_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('familia_0010.html')

### Juarez

In [ ]:
net_juarez_9000 = nx.Graph()
sXy_juarez_9000 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Juarez_J9000.txt')
Network_Generator_For_Cartels(net_juarez_9000,sXy_juarez_9000,df_J9000,'Juarez')

In [ ]:
nt = Network('1000px', '1000px')
nt.from_nx(net_juarez_9000, default_edge_weight=3,show_edge_weights=True)
nt.show('juarez_9000.html')

In [ ]:
all_weights=[]
for (node1,node2,data) in net_juarez_9000.edges(data=True):
        all_weights.append(data['weight']) 

nodes_color=[]
for (node1,data) in net_juarez_9000.nodes(data=True):
        nodes_color.append(data['color']) 


unique_weights = list(set(all_weights))

pos = nx.spring_layout(net_juarez_9000,seed=1234)

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in net_juarez_9000.edges(data=True) if edge_attr['weight']==weight]
        width = weight
        nx.draw_networkx_edges(net_juarez_9000,pos,edgelist=weighted_edges,width=width)

nx.draw_networkx_nodes(net_juarez_9000,pos,node_color=nodes_color, node_size=4000)
nx.draw_networkx_labels(net_juarez_9000,pos,font_size=15, font_family="sans-serif")
plt.rcParams["figure.figsize"] = (25,25)

pass

In [ ]:
metrics(net_juarez_9000,all_metrics=True)

In [ ]:
net_juarez_0010 = nx.Graph()
sXy_juarez_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Juarez_J0010.txt')
Network_Generator_For_Cartels(net_juarez_0010,sXy_juarez_0010,df_J0010,'Juarez')

In [ ]:
metrics(net_juarez_0010,all_metrics=True)

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_juarez_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('juarez_0010.html')

### Sinaloa

In [ ]:
net_sinaloa_9000 = nx.Graph()
sXy_sinaloa_9000 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Sinaloa_S9000.txt')
Network_Generator_For_Cartels(net_sinaloa_9000,sXy_sinaloa_9000,df_S9000,'Sinaloa')

In [ ]:
nt = Network('1000px', '1000px')
nt.from_nx(net_sinaloa_9000, default_edge_weight=3,show_edge_weights=True)
nt.show('sinaloa_9000.html')

In [ ]:
all_weights=[]
for (node1,node2,data) in net_sinaloa_9000.edges(data=True):
        all_weights.append(data['weight']) 

nodes_color=[]
for (node1,data) in net_sinaloa_9000.nodes(data=True):
        nodes_color.append(data['color']) 


unique_weights = list(set(all_weights))

pos = nx.spring_layout(net_sinaloa_9000,seed=1234)

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in net_sinaloa_9000.edges(data=True) if edge_attr['weight']==weight]
        width = weight
        nx.draw_networkx_edges(net_sinaloa_9000,pos,edgelist=weighted_edges,width=width)

nx.draw_networkx_nodes(net_sinaloa_9000,pos,node_color=nodes_color, node_size=4000)
nx.draw_networkx_labels(net_sinaloa_9000,pos,font_size=15, font_family="sans-serif")
plt.rcParams["figure.figsize"] = (25,25)

pass

In [ ]:
metrics(net_sinaloa_9000,all_metrics=True)

In [ ]:
net_sinaloa_0010 = nx.Graph()
sXy_sinaloa_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Sinaloa_S0010.txt')
Network_Generator_For_Cartels(net_sinaloa_0010,sXy_sinaloa_0010,df_S0010,'Sinaloa')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_sinaloa_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('sinaloa_0010.html')

In [ ]:
metrics(net_sinaloa_0010,all_metrics=True)

### Sinaloa Family

In [ ]:
net_sf_0010 = nx.Graph()
sXy_sf_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Sinaloa_Family_SF0010.txt')
Network_Generator_For_Cartels(net_sf_0010,sXy_sf_0010,df_SF0010,'Sinaloa_Family')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_sf_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('sf_0010.html')

### Tijuana

In [ ]:
net_tijuana_9000 = nx.Graph()
sXy_tijuana_9000 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Tijuana_T9000.txt')
Network_Generator_For_Cartels(net_tijuana_9000,sXy_tijuana_9000,df_T9000,'Tijuana')

In [ ]:
nt = Network('1000px', '1000px')
nt.from_nx(net_tijuana_9000, default_edge_weight=3,show_edge_weights=True)
nt.show('tijuana_9000.html')

In [ ]:
all_weights=[]
for (node1,node2,data) in net_tijuana_9000.edges(data=True):
        all_weights.append(data['weight']) 

nodes_color=[]
for (node1,data) in net_tijuana_9000.nodes(data=True):
        nodes_color.append(data['color']) 


unique_weights = list(set(all_weights))

pos = nx.spring_layout(net_tijuana_9000,seed=1234)

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in net_tijuana_9000.edges(data=True) if edge_attr['weight']==weight]
        width = weight
        nx.draw_networkx_edges(net_tijuana_9000,pos,edgelist=weighted_edges,width=width)

nx.draw_networkx_nodes(net_tijuana_9000,pos,node_color=nodes_color, node_size=4000)
nx.draw_networkx_labels(net_tijuana_9000,pos,font_size=15, font_family="sans-serif")
plt.rcParams["figure.figsize"] = (25,25)

pass

In [ ]:
metrics(net_tijuana_9000,all_metrics=True)

In [ ]:
net_tijuana_0010 = nx.Graph()
sXy_tijuana_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Tijuana_T0010.txt')
Network_Generator_For_Cartels(net_tijuana_0010,sXy_tijuana_0010,df_T0010,'Tijuana')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_tijuana_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('tijuana_0010.html')

In [ ]:
metrics(net_tijuana_0010,all_metrics=True)

### Zetas

In [ ]:
net_zetas_9000 = nx.Graph()
sXy_zetas_9000 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Zetas_Z9000.txt')
Network_Generator_For_Cartels(net_zetas_9000,sXy_zetas_9000,df_Z9000,'Zetas')

In [ ]:
nt = Network('1000px', '1000px')
nt.from_nx(net_zetas_9000, default_edge_weight=3,show_edge_weights=True)
nt.show('zetas_9000.html')

In [ ]:
all_weights=[]
for (node1,node2,data) in net_zetas_9000.edges(data=True):
        all_weights.append(data['weight']) 

nodes_color=[]
for (node1,data) in net_zetas_9000.nodes(data=True):
        nodes_color.append(data['color']) 


unique_weights = list(set(all_weights))

pos = nx.spring_layout(net_zetas_9000,seed=1234)

for weight in unique_weights:
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in net_zetas_9000.edges(data=True) if edge_attr['weight']==weight]
        width = weight
        nx.draw_networkx_edges(net_zetas_9000,pos,edgelist=weighted_edges,width=width)

nx.draw_networkx_nodes(net_zetas_9000,pos,node_color=nodes_color, node_size=4000)
nx.draw_networkx_labels(net_zetas_9000,pos,font_size=15, font_family="sans-serif")
plt.rcParams["figure.figsize"] = (25,25)

pass

In [ ]:
metrics(net_zetas_9000,all_metrics=True)

In [ ]:
net_zetas_0010 = nx.Graph()
sXy_zetas_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_Zetas_Z0010.txt')
Network_Generator_For_Cartels(net_zetas_0010,sXy_zetas_0010,df_Z0010,'Zetas')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_zetas_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('zetas_0010.html')

In [ ]:
metrics(net_zetas_0010,all_metrics=True)

### Otros

In [ ]:
net_otros_0010 = nx.Graph()
sXy_otros_0010 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_otros_O0010.txt')
Network_Generator_For_Cartels(net_otros_0010,sXy_otros_0010,df_O0010,'Otros')

In [ ]:
nt = Network('1000px','1000px')

nt.from_nx(net_otros_0010)

neighbor_map = nt.get_adj_list()
nt.repulsion(node_distance=200, spring_length=80)

for node in nt.nodes:
    node['value'] = len(neighbor_map[node['id']])

nt.show_buttons(filter_=True)
nt.show('otros_0010.html')

## NETWORK FAMIGLIE

In [ ]:
y_1992 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1992','network_cartels_1992',1992)
y_1993 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1993','network_cartels_1993',1993)
y_1994 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1994','network_cartels_1994',1994)
y_1995 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1995','network_cartels_1995',1995)
y_1996 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1996','network_cartels_1996',1996)
y_1997 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1997','network_cartels_1997',1997)
y_1998 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1998','network_cartels_1998',1998)
y_1999 = Edges_Only_Cartel_Network_Final(dta_9000,'network_cartels_1999','network_cartels_1999',1999)

y_9000 = pd.concat([y_1992,y_1993,y_1994,y_1995,y_1996,y_1997,y_1998,y_1999])

y_2000 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2000','network_cartels_2000',2000)
y_2001 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2001','network_cartels_2001',2001)
y_2002 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2002','network_cartels_2002',2002)
y_2003 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2003','network_cartels_2003',2003)
y_2004 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2004','network_cartels_2004',2004)
y_2005 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2005','network_cartels_2005',2005)
y_2006 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2006','network_cartels_2006',2006)
y_2007 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2007','network_cartels_2007',2007)
y_2008 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2008','network_cartels_2008',2008)
y_2009 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2009','network_cartels_2009',2009)
y_2010 = Edges_Only_Cartel_Network_Final(dta_0010,'network_cartels_2010','network_cartels_2010',2010)

y_0010 = pd.concat([y_2000,y_2001,y_2002,y_2003,y_2004,y_2005,y_2006,y_2007,y_2008,y_2009,y_2010])



In [ ]:
bi_grams_9000 = BiGram_Generator_Counter(y_9000)
bi_grams_0010 = BiGram_Generator_Counter(y_0010)

In [ ]:
bi_grams_9000.sort_values(by='W',ascending=False)

In [ ]:
bi_grams_0010.sort_values(by='W',ascending=False)

In [ ]:
a = bi_grams_0010[['A','W']].groupby(['A']).sum()

In [ ]:
b = bi_grams_0010[['B','W']].groupby(['B']).sum()

In [ ]:
a.to_csv('a.csv')
b.to_csv('b.csv')

In [ ]:
a = pd.read_csv('a.csv')
b = pd.read_csv('b.csv')

a.rename(columns = {'A':'DTO'}, inplace = True)
b.rename(columns = {'B':'DTO'}, inplace = True)

In [ ]:
ff = [a,b]
f = pd.concat(ff)
f = f.groupby('DTO').sum().sort_values(by='W',ascending=False)
f.to_csv('f.csv')
f = pd.read_csv('f.csv')

In [ ]:
f

In [ ]:
net_9000 = nx.Graph()
Network_Cartels_Meeting_Generator(bi_grams_9000,net_9000,'#0ee5ba')

net_0010 = nx.Graph()
Network_Cartels_Meeting_Generator(bi_grams_0010,net_0010,'#0ee5ba')

In [ ]:
options = {
    "node_color" : '#0ee5ba', 
    "node_size" : 4000,
    "font_color" : 'black', 
    "font_size" : 16
}

pos=nx.circular_layout(net_9000)
all_weights=[]
for (node1,node2,data) in net_9000.edges(data=True):
        all_weights.append(data['weight']) #we'll use this when determining edge thickness
unique_weights = list(set(all_weights))
for weight in unique_weights:
        #4 d. Form a filtered list with just the weight you want to draw
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in net_9000.edges(data=True) if edge_attr['weight']==weight]
        width = weight
        nx.draw_networkx_edges(net_9000,pos,edgelist=weighted_edges,width=width)
nx.draw(net_9000, pos, with_labels=True,**options)
plt.rcParams["figure.figsize"] = (15,10)
plt.show()

In [ ]:
metrics(net_9000, all_metrics=True)

In [ ]:
nx.betweenness_centrality(net_9000, weight='weight')

In [ ]:
tante = [(u, v) for (u, v, d) in net_0010.edges(data=True) if d["weight"] > 100 and d["weight"] < 150]
poche = [(u, v) for (u, v, d) in net_0010.edges(data=True) if d["weight"] <= 100 and d["weight"] > 50]
lot = [(u, v) for (u, v, d) in net_0010.edges(data=True) if d["weight"] >= 150]
non_troppe = [(u, v) for (u, v, d) in net_0010.edges(data=True) if d["weight"] <= 50]

#pos = nx.circular_layout(net_0010)
#nx.draw_networkx_nodes(G,pos, node_color= '#0ee5ba', node_size=4000)
# edges
nx.draw_networkx_edges(net_0010, pos, edgelist=tante, width=6,edge_color="#e5b31c")
nx.draw_networkx_edges(net_0010, pos, edgelist=poche, width=6 , edge_color="#4bf542")
nx.draw_networkx_edges(net_0010, pos, edgelist=lot, width=6, edge_color='#e70606' )
nx.draw_networkx_edges(net_0010, pos, edgelist=non_troppe, width=6,alpha=0.5, edge_color='#909090')

nx.draw(net_0010, pos, with_labels=True,**options)

plt.rcParams["figure.figsize"] = (15,10)
plt.show()

In [ ]:
metrics(net_0010, all_metrics=True)

In [ ]:
non_troppe[0][0]

In [ ]:
pp = net_0010

for i in range(len(non_troppe)):

    for ii in non_troppe[i]:
        pp.remove_edge(ii[0],ii[1])